# Build a semantic search engine with LangChain

## Overview

This tutorial will familiarize you with LangChain's [document loader](https://docs.langchain.com/oss/python/langchain/retrieval#document-loaders), [embedding](https://docs.langchain.com/oss/python/langchain/retrieval#embedding-models), and [vector store](https://docs.langchain.com/oss/python/langchain/retrieval#vector-store) abstractions. These abstractions are designed to support retrieval of data--  from (vector) databases and other sources -- for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation, or [RAG](https://docs.langchain.com/oss/python/langchain/retrieval).

Here we will build a search engine over a PDF document. This will allow us to retrieve passages in the PDF that are similar to an input query. The guide also includes a minimal RAG implementation on top of the search engine.

### Concepts

This guide focuses on retrieval of text data. We will cover the following concepts:

* [Documents and document loaders](https://docs.langchain.com/oss/python/integrations/document_loaders);
* [Text splitters](https://docs.langchain.com/oss/python/integrations/splitters);
* [Embeddings](https://docs.langchain.com/oss/python/integrations/text_embedding);
* [Vector stores](https://docs.langchain.com/oss/python/integrations/vectorstores) and [retrievers](https://docs.langchain.com/oss/python/integrations/retrievers).

## Setup

### Installation

This tutorial requires the `langchain-community` and `pypdf` packages:

In [3]:
%uv pip install langchain-community~=0.4 pypdf~=6.5

Using Python 3.13.11 environment at: /github.com/sammyne/langchain-tutorials/.venv
Audited 2 packages in 3ms
Note: you may need to restart the kernel to use updated packages.


## 1. Documents and Document Loaders

LangChain implements a [Document](https://reference.langchain.com/python/langchain_core/documents/#langchain_core.documents.base.Document) abstraction, which is intended to represent a unit of text and associated metadata. It has three attributes:

* `page_content`: a string representing the content;
* `metadata`: a dict containing arbitrary metadata;
* `id`: (optional) a string identifier for the document.

The `metadata` attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual [`Document`](https://reference.langchain.com/python/langchain_core/documents/#langchain_core.documents.base.Document) object often represents a chunk of a larger document.

We can generate sample documents when desired:


In [4]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]


However, the LangChain ecosystem implements [document loaders](https://docs.langchain.com/oss/python/langchain/retrieval#document-loaders) that [integrate with hundreds of common sources](https://docs.langchain.com/oss/python/integrations/document_loaders/). This makes it easy to incorporate data from these sources into your AI application.

### Loading documents

Let's load a PDF into a sequence of [`Document`](https://reference.langchain.com/python/langchain_core/documents/#langchain_core.documents.base.Document) objects. [Here is a sample PDF](./static/nke-10k-2023.pdf) -- a 10-k filing for Nike from 2023. We can consult the LangChain documentation for [available PDF document loaders](https://docs.langchain.com/oss/python/integrations/document_loaders/#pdfs).


In [5]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./static/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107



`PyPDFLoader` loads one [`Document`](https://reference.langchain.com/python/langchain_core/documents/#langchain_core.documents.base.Document) object per PDF page. For each, we can easily access:

* The string content of the page;
* Metadata containing the file name and page number.

In [6]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': './static/nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


### Splitting

For both information retrieval and downstream question-answering purposes, a page may be too coarse a representation. Our goal in the end will be to retrieve [`Document`](https://reference.langchain.com/python/langchain_core/documents/#langchain_core.documents.base.Document) objects that answer an input query, and further splitting our PDF will help ensure that the meanings of relevant portions of the document are not "washed out" by surrounding text.

We can use [text splitters](https://docs.langchain.com/oss/python/langchain/retrieval#text_splitters) for this purpose. Here we will use a simple text splitter that partitions based on characters. We will split our documents into chunks of 1000 characters
with 200 characters of overlap between chunks. The overlap helps
mitigate the possibility of separating a statement from important
context related to it. We use the
`RecursiveCharacterTextSplitter`,
which will recursively split the document using common separators like
new lines until each chunk is the appropriate size. This is the
recommended text splitter for generic text use cases.

We set `add_start_index=True` so that the character index where each
split Document starts within the initial Document is preserved as
metadata attribute “start\_index”.

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(len(all_splits))

516



## 2. Embeddings

Vector search is a common way to store and search over unstructured data (such as unstructured text). The idea is to store numeric vectors that are associated with the text. Given a query, we can [embed](https://docs.langchain.com/oss/python/langchain/retrieval#embedding_models) it as a vector of the same dimension and use vector similarity metrics (such as cosine similarity) to identify related text.

LangChain supports embeddings from [dozens of providers](https://docs.langchain.com/oss/python/integrations/text_embedding/). These models specify how text should be converted into a numeric vector. Let's select a model:

> 其他嵌入模型的集成参见 https://docs.langchain.comhttps://docs.langchain.com/oss/python/langchain/knowledge-base#2-embeddings


In [12]:
# 安装 CPU 版 PyTorch，避免 sentence-transformers 自动安装 GPU 版 PyTorch
%uv pip install torch~=2.9 --index-url https://download.pytorch.org/whl/cpu

Using Python 3.13.11 environment at: /github.com/sammyne/langchain-tutorials/.venv
Audited 1 package in 2ms
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%uv pip install langchain-huggingface~=1.2 sentence-transformers~=5.2

Note: you may need to restart the kernel to use updated packages.


In [17]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
# model_name='BAAI/bge-small-en-v1.5'
embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [18]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[0.047472212463617325, 0.021675797179341316, -0.009018072858452797, 0.005356720648705959, 0.02555766887962818, -0.010230258107185364, -0.008414018899202347, 0.039303913712501526, 0.021570539101958275, -0.024095432832837105]


Armed with a model for generating text embeddings, we can next store them in a special data structure that supports efficient similarity search.

## 3. Vector stores

LangChain [VectorStore](https://reference.langchain.com/python/langchain_core/vectorstores/?h=#langchain_core.vectorstores.base.VectorStore) objects contain methods for adding text and [`Document`](https://reference.langchain.com/python/langchain_core/documents/#langchain_core.documents.base.Document) objects to the store, and querying them using various similarity metrics. They are often initialized with [embedding](https://docs.langchain.com/oss/python/langchain/retrieval#embedding_models) models, which determine how text data is translated to numeric vectors.

LangChain includes a suite of [integrations](https://docs.langchain.com/oss/python/integrations/vectorstores) with different vector store technologies. Some vector stores are hosted by a provider (e.g., various cloud providers) and require specific credentials to use; some (such as [Postgres](https://docs.langchain.com/oss/python/integrations/vectorstores/pgvector)) run in separate infrastructure that can be run locally or via a third-party; others can run in-memory for lightweight workloads. Let's select a vector store:

In [20]:
%uv pip install langchain-chroma~=1.0

Using Python 3.13.11 environment at: /github.com/sammyne/langchain-tutorials/.venv
Audited 1 package in 7ms


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [21]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./_chroma.db",  # Where to save data locally, remove if not necessary
)

Having instantiated our vector store, we can now index the documents.

In [22]:
ids = vector_store.add_documents(documents=all_splits)

Note that most vector store implementations will allow you to connect to an existing vector store--  e.g., by providing a client, index name, or other information. See the documentation for a specific [integration](https://docs.langchain.com/oss/python/integrations/vectorstores) for more detail.

Once we've instantiated a [`VectorStore`](https://reference.langchain.com/python/langchain_core/vectorstores/?h=#langchain_core.vectorstores.base.VectorStore) that contains documents, we can query it. [VectorStore](https://reference.langchain.com/python/langchain_core/vectorstores/?h=#langchain_core.vectorstores.base.VectorStore) includes methods for querying:

* Synchronously and asynchronously;
* By string query and by vector;
* With and without returning similarity scores;
* By similarity and [maximum marginal relevance](https://reference.langchain.com/python/langchain_core/vectorstores/?h=#langchain_core.vectorstores.base.VectorStore.max_marginal_relevance_search) (to balance similarity with query to diversity in retrieved results).

The methods will generally include a list of [Document](https://reference.langchain.com/python/langchain_core/documents/#langchain_core.documents.base.Document) objects in their outputs.

**Usage**

Embeddings typically represent text as a "dense" vector such that texts with similar meanings are geometrically close. This lets us retrieve relevant information just by passing in a question, without knowledge of any specific key-terms used in the document.

Return documents based on similarity to a string query:


In [23]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our
wholesale, NIKE Direct and merchandising strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which are
leased. Two other distribution centers, one located in Indianapolis, Indiana and one located in Dayton, Tennessee, are leased and operated by third-party logistics
providers. One distribution center for Converse is located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States,
some of which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United States are located in Laakdal,' metadata={'title': '0000320187-23-000039', 'creationdate': '2

Async query:

In [24]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='Table of Contents
PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"
"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectively, unless the context indicates otherwise.
Our principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, equipment, accessories and services. NIKE is
the largest seller of athletic footwear and apparel in the world. We sell our products through NIKE Direct operations, which are comprised of both NIKE-owned retail stores
and sales through our digital platforms (also referred to as "NIKE Brand Digital"), to retail accounts and to a mix of independent distributors, licensees and sales' metadata={'source': './static/nke-10k-2023.pdf', 'author': 'EDGAR Online, a division of Donnelley Financia

Return scores:

In [25]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.37252241373062134

page_content='Table of Contents
YEAR ENDED MAY 31,
(Dollars in millions) 2023 2022 2021
REVENUES
North America $ 21,608 $ 18,353 $ 17,179 
Europe, Middle East & Africa 13,418 12,479 11,456 
Greater China 7,248 7,547 8,290 
Asia Pacific & Latin America 6,431 5,955 5,343 
Global Brand Divisions 58 102 25 
Total NIKE Brand 48,763 44,436 42,293 
Converse 2,427 2,346 2,205 
Corporate 27 (72) 40 
TOTAL NIKE, INC. REVENUES $ 51,217 $ 46,710 $ 44,538 
EARNINGS BEFORE INTEREST AND TAXES
North America $ 5,454 $ 5,114 $ 5,089 
Europe, Middle East & Africa 3,531 3,293 2,435 
Greater China 2,283 2,365 3,243 
Asia Pacific & Latin America 1,932 1,896 1,530 
Global Brand Divisions (4,841) (4,262) (3,656)
Converse 676 669 543 
Corporate (2,840) (2,219) (2,261)
Interest expense (income), net (6) 205 262 
TOTAL NIKE, INC. INCOME BEFORE INCOME TAXES $ 6,201 $ 6,651 $ 6,661 
ADDITIONS TO PROPERTY, PLANT AND EQUIPMENT
North America $ 283 $ 146 $ 98 
Europe, Middle East & Africa 2

Return documents based on similarity to an embedded query:

In [26]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Table of Contents
GROSS MARGIN
FISCAL 2023 COMPARED TO FISCAL 2022
For fiscal 2023, our consolidated gross profit increased 4% to $22,292 million compared to $21,479 million for fiscal 2022. Gross margin decreased 250 basis points to
43.5% for fiscal 2023 compared to 46.0% for fiscal 2022 due to the following:
*Wholesale equivalent
The decrease in gross margin for fiscal 2023 was primarily due to:
• Higher NIKE Brand product costs, on a wholesale equivalent basis, primarily due to higher input costs and elevated inbound freight and logistics costs as well as
product mix;
• Lower margin in our NIKE Direct business, driven by higher promotional activity to liquidate inventory in the current period compared to lower promotional activity in
the prior period resulting from lower available inventory supply;
• Unfavorable changes in net foreign currency exchange rates, including hedges; and
• Lower off-price margin, on a wholesale equivalent basis.
This was partially offset by:'

Learn more:

* [API Reference](https://reference.langchain.com/python/langchain_core/vectorstores/?h=#langchain_core.vectorstores.base.VectorStore)
* [Integration-specific docs](https://docs.langchain.com/oss/python/integrations/vectorstores)

## 4. Retrievers

LangChain [`VectorStore`](https://reference.langchain.com/python/langchain_core/vectorstores/?h=#langchain_core.vectorstores.base.VectorStore) objects do not subclass [Runnable](https://reference.langchain.com/python/langchain_core/runnables/#langchain_core.runnables.Runnable). LangChain [Retrievers](https://reference.langchain.com/python/langchain_core/retrievers/#langchain_core.retrievers.BaseRetriever) are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous `invoke` and `batch` operations). Although we can construct retrievers from vector stores, retrievers can interface with non-vector store sources of data, as well (such as external APIs).

We can create a simple version of this ourselves, without subclassing `Retriever`. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the `similarity_search` method:

In [27]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='c398e6f5-8faa-4f6e-bb64-19cc4e05ab96', metadata={'creator': 'EDGAR Filing HTML Converter', 'page_label': '27', 'start_index': 804, 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'title': '0000320187-23-000039', 'keywords': '0000320187-23-000039; ; 10-K', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'moddate': '2023-07-20T16:22:08-04:00', 'creationdate': '2023-07-20T16:22:00-04:00', 'source': './static/nke-10k-2023.pdf', 'total_pages': 107, 'page': 26, 'author': 'EDGAR Online, a division of Donnelley Financial Solutions'}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three

Vectorstores implement an `as_retriever` method that will generate a Retriever, specifically a [VectorStoreRetriever](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html). These retrievers include specific `search_type` and `search_kwargs` attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:


In [28]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='c398e6f5-8faa-4f6e-bb64-19cc4e05ab96', metadata={'moddate': '2023-07-20T16:22:08-04:00', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'start_index': 804, 'creator': 'EDGAR Filing HTML Converter', 'total_pages': 107, 'page_label': '27', 'creationdate': '2023-07-20T16:22:00-04:00', 'keywords': '0000320187-23-000039; ; 10-K', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'page': 26, 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'source': './static/nke-10k-2023.pdf'}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three

`VectorStoreRetriever` supports search types of `"similarity"` (default), `"mmr"` (maximum marginal relevance, described above), and `"similarity_score_threshold"`. We can use the latter to threshold documents output by the retriever by similarity score.

Retrievers can easily be incorporated into more complex applications, such as [retrieval-augmented generation (RAG)](https://docs.langchain.com/oss/python/langchain/retrieval) applications that combine a given question with retrieved context into a prompt for a LLM. To learn more about building such an application, check out the [RAG tutorial](https://docs.langchain.com/oss/python/langchain/rag) tutorial.

## Next steps

You've now seen how to build a semantic search engine over a PDF document.

For more on document loaders:

* [Overview](https://docs.langchain.com/oss/python/langchain/retrieval#document_loaders)
* [Available integrations](https://docs.langchain.com/oss/python/integrations/document_loaders/)

For more on embeddings:

* [Overview](https://docs.langchain.com/oss/python/langchain/retrieval#embedding_models/)
* [Available integrations](https://docs.langchain.com/oss/python/integrations/text_embedding/)

For more on vector stores:

* [Overview](https://docs.langchain.com/oss/python/langchain/retrieval#vectorstores/)
* [Available integrations](https://docs.langchain.com/oss/python/integrations/vectorstores/)

For more on RAG, see:

* [Build a Retrieval Augmented Generation (RAG) App](https://docs.langchain.com/oss/python/langchain/rag/)